In [12]:
import os
import pandas as pd

# remove or add teams to the list you want to process.
team_list = [
    "atlanta-united", 
    "austin-fc", 
    "CF-Montreal", 
    "Charlotte-FC", 
    "chicago-fire", 
    "colorado-rapids", 
    "Columbus-Crew", 
    "dc-united", 
    "fc-cincinnati",
    "fc-dallas", 
    "houston-dynamo",
    "inter-miami", 
    "la-galaxy", 
    "los-angeles-fc", 
    "minnesota-united",
    "Nashville-SC", 
    "New-England-Revolution", 
    "New-York-City-FC", 
    "new-york-red-bulls",
    "orlando-city", 
    "Philadelphia-Union", 
    "Portland-Timbers", 
    "Real-Salt-Lake", 
    "San-Jose-Earthquakes",
    "Seattle-Sounders-FC", 
    "Sporting-Kansas", 
    "st-louis-city", 
    "Toronto-FC", 
    "Vancouver-Whitecaps"
]

base_folder = r"C:\Users\Yusuf\OneDrive - Georgia Institute of Technology\GT MSA\6414\Regression Team Project\MLS_2024_Data" # set working directory as needed.

all_team_dict = {} # dict to store all processed team data. Format is (team name:team master df) as key:value pair
 
prefixes = ["def_", "g&s_", "keeper_", "misc_", "pass_", "passtype_", "poss_", "shoot_"]  # adjust this list as needed

for team in team_list:
    folder_path = os.path.join(base_folder, team) 
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')] # gets a list of all the csv for a team
    
    dataframes = []  # will stores all dataframes for this team
    
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path) 
        dataframes.append(df)
        # print(f"Loaded from {team}: {file}, Shape: {df.shape}")
    
    if len(dataframes) <= 7:  # Less than 8 CSV files
        print(f"Skipping {team}: Not enough files (found {len(dataframes)}).")
        continue  # Skip to the next team
    
    scores_fixtures = dataframes[7]  # master dataframe for this team
    
    scores_fixtures = scores_fixtures.iloc[:, :-2]  # filter out Match Report and Notes columns
    
    filtered_dataframes = [df for i, df in enumerate(dataframes) if i != 7] # remove score_fixtures from list of df
    
    processed_dataframes = []
    for df, prefix in zip(filtered_dataframes, prefixes):
        if 'Match Report' in df.columns:
            df = df.drop(columns=['Match Report'])  # drop the Match Report column
        
        df = df.iloc[:, 9:].add_prefix(prefix)  # add prefix and select columns from index 9 onward
        processed_dataframes.append(df)
    
    master = pd.concat([scores_fixtures] + processed_dataframes, axis=1)
    
    if 'Round' not in master.columns:
        print(f"Skipping {team}: 'Round' column not found in data.")
        continue  # Skip this team if 'Round' is missing.
    
    mask = (master['Round'] == 'Regular Season') # this applies the "Regular Season" filter
    master = master[mask]
    
    master.insert(0, 'Team', team)
    
    all_team_dict[team] = master


atlanta_master = all_team_dict["atlanta-united"] # getting the master of each team
# display(atlanta_master)

all_teams_master = pd.concat(all_team_dict.values(), axis=0, ignore_index=True) # concatenate all team masters together
display(all_teams_master)

,Team,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,...,shoot_G/SoT,shoot_Dist,shoot_FK,shoot_PK,shoot_PKatt,shoot_xG,shoot_npxG,shoot_npxG/Sh,shoot_G-xG,shoot_np:G-xG
0,atlanta-united,2024-02-24,14:00,Regular Season,Sat,Away,L,0,1,Columbus Crew,...,0.00,20.0,0.0,0.0,0.0,0.5,0.5,0.06,-0.5,-0.5
1,atlanta-united,2024-03-09,19:30,Regular Season,Sat,Home,W,4,1,NE Revolution,...,0.29,18.8,0.0,2.0,2.0,3.2,1.6,0.09,0.8,0.4
2,atlanta-united,2024-03-17,19:00,Regular Season,Sun,Home,W,2,0,Orlando City,...,0.29,15.3,0.0,0.0,0.0,0.8,0.8,0.07,1.2,1.2
3,atlanta-united,2024-03-23,19:30,Regular Season,Sat,Away,L,0,2,Toronto FC,...,0.00,16.7,0.0,0.0,0.0,0.4,0.4,0.05,-0.4,-0.4
4,atlanta-united,2024-03-31,15:30,Regular Season,Sun,Home,W,3,0,Chicago Fire,...,0.43,13.6,0.0,0.0,0.0,2.7,2.7,0.13,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,Vancouver-Whitecaps,2024-09-28,19:30,Regular Season,Sat,Home,D,1.0,1.0,Portland Timbers,...,0.33,18.9,0.0,0.0,0.0,1.0,1.0,0.11,0.0,0.0
982,Vancouver-Whitecaps,2024-10-02,19:30,Regular Season,Wed,Home,L,0.0,3.0,Seattle Sounders FC,...,0.00,14.3,1.0,0.0,0.0,1.0,1.0,0.11,-1.0,-1.0
983,Vancouver-Whitecaps,2024-10-05,16:30,Regular Season,Sat,Home,L,0.0,1.0,Minnesota Utd,...,0.00,15.0,1.0,0.0,0.0,0.5,0.5,0.05,-0.5,-0.5
984,Vancouver-Whitecaps,2024-10-13,16:30,Regular Season,Sun,Home,L,1.0,2.0,LAFC,...,0.00,17.6,0.0,0.0,0.0,1.1,1.1,0.07,-1.1,-1.1


In [13]:
import os
import pandas as pd

base_folder = r"C:\Users\Yusuf\OneDrive - Georgia Institute of Technology\GT MSA\6414\Regression Team Project\MLS_2024_Data"

all_teams_data = [] # this initializes a list to store the data

for team_folder in os.listdir(base_folder): # loops through all of the team folders
    team_path = os.path.join(base_folder, team_folder)

    if os.path.isdir(team_path):
        keeper_path = os.path.join(team_path, 'keeper_data.csv')
        defense_path = os.path.join(team_path, 'defense_data.csv')

        if os.path.exists(keeper_path) and os.path.exists(defense_path): # checks if the paths exists
            keeper_data = pd.read_csv(keeper_path) # this creates the keeper df
            defense_data = pd.read_csv(defense_path) # this creates the defense df
            keeper_data = keeper_data.add_prefix('opp_keeper_') # add the prefix opp def 
            keeper_data['Team'] = team_folder
            defense_data = defense_data.add_prefix('opp_def_') # add the prefix opp def 
            defense_data['Team'] = team_folder
            keeper_data = keeper_data.rename(columns={'opp_keeper_Date':'Date'}) 
            defense_data = defense_data.rename(columns={'opp_def_Date':'Date'})
            
            team_data = keeper_data.merge(defense_data, on = ['Team','Date'], how = 'left') # merges the keeper df and defense df together
            
            all_teams_data.append(team_data) # appends the merged data for that team to the list of all teams

opp_data = pd.concat(all_teams_data, ignore_index=True) # concatenates the list to a dataframe called opp

display(opp_data.head(3))

,Date,opp_keeper_Time,opp_keeper_Round,opp_keeper_Day,opp_keeper_Venue,opp_keeper_Result,opp_keeper_GF,opp_keeper_GA,opp_keeper_Opponent,opp_keeper_SoTA,...,opp_def_Tkl%,opp_def_Lost,opp_def_Blocks,opp_def_Sh,opp_def_Pass,opp_def_Int,opp_def_Tkl+Int,opp_def_Clr,opp_def_Err,opp_def_Match Report
0,2024-02-24,14:00,Regular Season,Sat,Away,L,0,1,Columbus Crew,3.0,...,62.5,9.0,18.0,6.0,12.0,9.0,32.0,15.0,1.0,Match Report
1,2024-03-09,19:30,Regular Season,Sat,Home,W,4,1,NE Revolution,5.0,...,37.5,5.0,6.0,2.0,4.0,7.0,23.0,9.0,0.0,Match Report
2,2024-03-17,19:00,Regular Season,Sun,Home,W,2,0,Orlando City,3.0,...,43.5,13.0,12.0,3.0,9.0,7.0,21.0,30.0,0.0,Match Report


In [14]:
opp_data = opp_data[opp_data['opp_keeper_Round'] == 'Regular Season'] # this masks and filters out games that are only from the regular season
opp_data.count()

Date                    986
opp_keeper_Time         986
opp_keeper_Round        986
opp_keeper_Day          986
opp_keeper_Venue        986
                       ... 
opp_def_Int             986
opp_def_Tkl+Int         986
opp_def_Clr             986
opp_def_Err             986
opp_def_Match Report    986
Length: 62, dtype: int64

In [15]:
opp_data.columns # check the columns for the ones to drop

Index(['Date', 'opp_keeper_Time', 'opp_keeper_Round', 'opp_keeper_Day',
       'opp_keeper_Venue', 'opp_keeper_Result', 'opp_keeper_GF',
       'opp_keeper_GA', 'opp_keeper_Opponent', 'opp_keeper_SoTA',
       'opp_keeper_GA.1', 'opp_keeper_Saves', 'opp_keeper_Save%',
       'opp_keeper_CS', 'opp_keeper_PSxG', 'opp_keeper_PSxG+/-',
       'opp_keeper_PKatt', 'opp_keeper_PKA', 'opp_keeper_PKsv',
       'opp_keeper_PKm', 'opp_keeper_Cmp', 'opp_keeper_Att', 'opp_keeper_Cmp%',
       'opp_keeper_Att (GK)', 'opp_keeper_Thr', 'opp_keeper_Launch%',
       'opp_keeper_AvgLen', 'opp_keeper_Att.1', 'opp_keeper_Launch%.1',
       'opp_keeper_AvgLen.1', 'opp_keeper_Opp', 'opp_keeper_Stp',
       'opp_keeper_Stp%', 'opp_keeper_#OPA', 'opp_keeper_AvgDist',
       'opp_keeper_Match Report', 'Team', 'opp_def_Time', 'opp_def_Round',
       'opp_def_Day', 'opp_def_Venue', 'opp_def_Result', 'opp_def_GF',
       'opp_def_GA', 'opp_def_Opponent', 'opp_def_Tkl', 'opp_def_TklW',
       'opp_def_Def 3rd', 'op

In [16]:
dropped_columns = ['opp_def_Time', 'opp_def_Round','opp_def_Day', 'opp_def_Venue', 'opp_def_Result', 'opp_def_GF',
                   'opp_def_GA', 'opp_def_Match Report', 'opp_keeper_Match Report',  'opp_keeper_Time', 'opp_keeper_Round', 'opp_keeper_Day',
                   'opp_keeper_Venue', 'opp_keeper_Result', 'opp_keeper_GF', 'opp_keeper_GA', 'opp_def_Opponent','opp_keeper_Opponent']

opp_data = opp_data.drop(columns=dropped_columns)

In [17]:
print(opp_data.columns)
opp_data = opp_data.rename(columns={'Team':'Opponent'}) 
# we want to name Team to Opponent here. This allows us to merge on Opponent later when we want to merge all_master_team and opp_data.

Index(['Date', 'opp_keeper_SoTA', 'opp_keeper_GA.1', 'opp_keeper_Saves',
       'opp_keeper_Save%', 'opp_keeper_CS', 'opp_keeper_PSxG',
       'opp_keeper_PSxG+/-', 'opp_keeper_PKatt', 'opp_keeper_PKA',
       'opp_keeper_PKsv', 'opp_keeper_PKm', 'opp_keeper_Cmp', 'opp_keeper_Att',
       'opp_keeper_Cmp%', 'opp_keeper_Att (GK)', 'opp_keeper_Thr',
       'opp_keeper_Launch%', 'opp_keeper_AvgLen', 'opp_keeper_Att.1',
       'opp_keeper_Launch%.1', 'opp_keeper_AvgLen.1', 'opp_keeper_Opp',
       'opp_keeper_Stp', 'opp_keeper_Stp%', 'opp_keeper_#OPA',
       'opp_keeper_AvgDist', 'Team', 'opp_def_Tkl', 'opp_def_TklW',
       'opp_def_Def 3rd', 'opp_def_Mid 3rd', 'opp_def_Att 3rd',
       'opp_def_Tkl.1', 'opp_def_Att', 'opp_def_Tkl%', 'opp_def_Lost',
       'opp_def_Blocks', 'opp_def_Sh', 'opp_def_Pass', 'opp_def_Int',
       'opp_def_Tkl+Int', 'opp_def_Clr', 'opp_def_Err'],
      dtype='object')


In [18]:
# Currently, the "Team" column name is inconsistent to the Opponent column name. Let's fix that
all_teams_master['Opponent'] = all_teams_master['Opponent'].astype(str)
opp_set = set(all_teams_master['Opponent'])
opp_list = sorted(opp_set)
 
team_mapping = dict(zip(team_list, opp_list)) # this maps the team_list from the beginning to opp_list

# after debugging, we found that our mapping was incorrect for a few teams. The code below fixes that.
team_mapping['new-york-red-bulls'] = 'NY Red Bulls'
team_mapping['New-England-Revolution'] ='NE Revolution'
team_mapping['Nashville-SC'] = 'Nashville SC'

# Replace the team names in the 'Team' column using the mapping
all_teams_master['Team'] = all_teams_master['Team'].replace(team_mapping)
opp_data['Opponent'] = opp_data['Opponent'].replace(team_mapping)

In [19]:
# this merges all_teams_master with opp_data using a left join on Opponent, Date
master_mls_data = all_teams_master.merge(opp_data, on = ['Opponent','Date'], how = 'left')

In [20]:
# we need to drop the original defense and keeper tables from all_master_teams
columns_to_drop = [
    col for col in master_mls_data.columns 
    if (col.startswith('keeper_') or col.startswith('def_')) 
    and not (col.startswith('opp_keeper') or col.startswith('opp_def'))
]

# Drop the identified columns
master_mls_data = master_mls_data.drop(columns=columns_to_drop)

In [21]:
master_mls_playoff = master_mls_data.copy()